In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. 加载鸢尾花数据集
iris = load_iris()

# X：数据内容
X = iris.data
# y：数据标签（监督式学习）
y = iris.target

# 2. 划分训练集与测试集，80%+20%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. 标准化数据（对KNN非常重要，因为它基于距离）
# $注意： y_train（标签）不需要标准化。标准化通常只针对特征X进行，因为KNN等算法是基于特征空间的距离计算的，标签y只是分类的目标，不参与距离计算。
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 标准化训练集
X_test_scaled = scaler.transform(X_test)        # 标准化测试集


# 4. 初始化KNN模型，设置邻居数为5（n_neighbors=5），用于后续训练
knn = KNeighborsClassifier(n_neighbors=26)      # 邻居数量过大，会导致模型准确率下降

# 5. 用标准化后的训练集数据拟合KNN模型，完成模型训练
knn.fit(X_train_scaled, y_train)

# 5. 进行预测。predict方法返回的是测试集样本的预测类别标签，即每个样本被模型判定为哪一类。例如：[0 1 2 1 ...]
y_pred = knn.predict(X_test_scaled)

# 6. 模型评估
print("准确率：", accuracy_score(y_test, y_pred))
print("分类报告：\n", classification_report(y_test, y_pred, target_names=iris.target_names))


准确率： 0.9666666666666667
分类报告：
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.90      1.00      0.95         9
   virginica       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



## 采用网格搜索优化超参数

In [ ]:
# 加载数据，划分训练集和测试集，标准化数据，初始化KNN模型

# 定义模型和参数网格
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],    # uniform=相等权重，distance=距离加权
    'p': [1, 2]  # 1=曼哈顿距离, 2=欧几里得距离
}

# 使用 GridSearchCV 进行超参数搜索
# 创建GridSearchCV对象，传入KNN模型、参数网格，使用3折交叉验证，评估指标（标签)为准确率
grid_search = GridSearchCV(knn, param_grid, cv=3, scoring='accuracy')

# 在训练集上进行网格搜索，寻找最佳超参数组合
grid_search.fit(X_train_scaled, y_train)
print("最优参数：", grid_search.best_params_)
print("最优得分（训练集CV）：", grid_search.best_score_)

# 这里不需要再调用knn.fit，因为best_knn已经由GridSearchCV在fit过程中自动训练好了


# 用最优模型预测测试集
best_knn = grid_search.best_estimator_

# 这里可以直接用best_knn进行predict，是因为GridSearchCV在fit时已经用最优参数自动训练好了模型
y_pred = best_knn.predict(X_test_scaled)

# 评估模型
print("测试集准确率：", accuracy_score(y_test, y_pred))
print("分类报告：\n", classification_report(y_test, y_pred, target_names=iris.target_names))

最优参数： {'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
最优得分（训练集CV）： 0.9583333333333334
测试集准确率： 1.0
分类报告：
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

